In [ ]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

ROOT_PATH_DRIVE = "/content/drive/MyDrive/Năm 4/Kỳ 2/Xây dựng các hệ thống nhúng - T.Kiên/Project"
RECORD_PATH = os.path.join(ROOT_PATH_DRIVE, "coco_dataset", "tfrecord")
CUSTOM_MODEL_NAME = "ssd_mobilenet_model"
PRETRAINED_MODEL_NAME = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
PRETRAINED_MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
TF_RECORD_SCRIPT_NAME = "generate_tfrecord.py"
LABEL_MAP_NAME = "label_map.pbtxt"

paths = {
    "WORKSPACE_PATH": os.path.join("Tensorflow", "workspace"),
    "SCRIPTS_PATH": os.path.join("Tensorflow", "scripts"),
    "APIMODEL_PATH": os.path.join("Tensorflow", "models"),
    "ANNOTATION_PATH": os.path.join("Tensorflow", "workspace", "annotations"),
    "MODEL_PATH": os.path.join("Tensorflow", "workspace", "models"),
    "PRETRAINED_MODEL_PATH": os.path.join("Tensorflow", "workspace", "pre-trained-models"),
    "MY_MODEL_PATH": os.path.join("Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME),
    "CHECKPOINT_PATH": os.path.join("Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "checkpoint"),
    "OUTPUT_PATH": os.path.join("Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "export"),
    "TFJS_PATH": os.path.join("Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "tfjsexport"),
    "TFLITE_PATH": os.path.join("Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "tfliteexport"),
    "PROTOC_PATH": os.path.join("Tensorflow", "protoc"),
}

files = {
    "PIPELINE_CONFIG": os.path.join(paths["MY_MODEL_PATH"], "pipeline.config"),
    "TF_RECORD_SCRIPT": os.path.join(paths["SCRIPTS_PATH"], TF_RECORD_SCRIPT_NAME),
    "LABELMAP": os.path.join(ROOT_PATH_DRIVE, "label_map.pbtxt"),
    "TRAIN_RECORD": os.path.join(RECORD_PATH, "train", "train.tfrecord"),
    "VAL_RECORD": os.path.join(RECORD_PATH, "val", "val.tfrecord"),
}


In [ ]:
# Creating workspace folder
for path in paths.values():
  if not os.path.exists(path):
    if os.name == 'posix':
      !mkdir -p {path}

In [ ]:
# Clone API
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow object detection and compile these
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

In [ ]:
# Verify installation
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

In [ ]:
# Download pre-trained model
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [ ]:
# Get pipeline.config
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['MY_MODEL_PATH'])}

In [ ]:
# Config pipeline.config
FINE_TUNE_CP_FILE = os.path.join(paths["PRETRAINED_MODEL_PATH"], PRETRAINED_MODEL_NAME, "checkpoint", "ckpt-0")

config = config_util.get_configs_from_pipeline_file(files["PIPELINE_CONFIG"])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

with tf.io.gfile.GFile(files["PIPELINE_CONFIG"], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = 16  # 16 labels
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = FINE_TUNE_CP_FILE  # restore checkpoint
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"  # type train
pipeline_config.train_input_reader.label_map_path = files["LABELMAP"]
pipeline_config.eval_input_reader[0].label_map_path = files["LABELMAP"]

pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [files["TRAIN_RECORD"]]  # TFRecord Train file
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    files["VAL_RECORD"]
]  # TFRecord Validation file

config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files["PIPELINE_CONFIG"], "wb") as f:
    f.write(config_text)


In [ ]:
# Train model
TRAINING_SCRIPT = os.path.join(paths["APIMODEL_PATH"], "research", "object_detection", "model_main_tf2.py")
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(
    TRAINING_SCRIPT, paths["MY_MODEL_PATH"], files["PIPELINE_CONFIG"]
)
!{command}


In [ ]:
# Evaluation model
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(
    TRAINING_SCRIPT, paths["MY_MODEL_PATH"], files["PIPELINE_CONFIG"], paths["CHECKPOINT_PATH"]
)
